In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [52]:
# load in excel sheets
xls = pd.ExcelFile('Basketball_dataset_clean_2.xlsx')
final_records = pd.read_excel(xls, 0)
game_records = pd.read_excel(xls, list(range(1,69)))

In [53]:
# calculate score differential for all games
for i in range(1,69):
    game_records[i]['Opponent'] = game_records[i]['Opponent'].str.replace(r'\(\d+\)', '', regex=True)
    game_records[i]['Opponent'] = game_records[i]['Opponent'].str.strip()
    game_records[i]['Score_diff'] = game_records[i]['Tm'] - game_records[i]['Opp']
    game_records[i]['Score_diff_norm'] = game_records[i]['Score_diff'] / (game_records[i]['Tm'] + game_records[i]['Opp']) 
    game_records[i]['Score_diff_ratio'] = game_records[i]['Tm'] / game_records[i]['Opp']
    game_records[i]['Score_diff_ratio'] = np.where(game_records[i]['Score_diff_ratio'] > 1, game_records[i]['Score_diff_ratio'], np.reciprocal(game_records[i]['Score_diff_ratio']) )



In [54]:
print(game_records[13])

     G             Date    Type          Opponent W/L   Tm  Opp  Score_diff  \
0    1   Mon Nov 6 2023     REG         Dartmouth   W   92   54          38   
1    2  Fri Nov 10 2023     REG           Arizona   L   73   78          -5   
2    3  Tue Nov 14 2023     REG    Michigan State   W   74   65           9   
3    4  Fri Nov 17 2023     REG          Bucknell   W   90   60          30   
4    5  Tue Nov 21 2023     REG          La Salle   W   95   66          29   
5    6  Fri Nov 24 2023     REG  Southern Indiana   W   80   62          18   
6    7  Wed Nov 29 2023     REG          Arkansas   L   75   80          -5   
7    8   Sat Dec 2 2023     REG      Georgia Tech   L   68   72          -4   
8    9   Sat Dec 9 2023     REG         Charlotte   W   80   56          24   
9   10  Tue Dec 12 2023     REG           Hofstra   W   89   68          21   
10  11  Wed Dec 20 2023     REG            Baylor   W   78   70           8   
11  12  Sat Dec 30 2023     REG       Queens (NC)   

In [55]:
# Split the 'Record' column into 'Wins' and 'Losses'
final_records[['Wins', 'Losses']] = final_records['Record'].str.split('-', expand=True).astype(int)

# calculate final_records stats
#final_records["Score_diff"] = np.NaN
#final_records["Score_diff_norm"] = np.NaN
#for i in range(1,69):    
#    final_records["Score_diff"][i-1] = np.sum(game_records[i]['Score_diff'])
#    final_records["Score_diff_norm"][i-1] = np.sum(game_records[i]['Score_diff_norm'])

In [56]:
print(final_records)

    Seed            Team Record  Wins  Losses
0      1           UConn   31-3    31       3
1      2         Houston   30-4    30       4
2      3          Purdue   29-4    29       4
3      4  North Carolina   27-7    27       7
4      5       Tennessee   24-8    24       8
..   ...             ...    ...   ...     ...
63    64         Stetson  22-12    22      12
64    65     Montana St.  17-17    17      17
65    66       Grambling  20-14    20      14
66    67          Howard  18-16    18      16
67    68          Wagner  16-15    16      15

[68 rows x 5 columns]


In [57]:
#
#plt.plot(final_records["Seed"],final_records["Score_diff"])
#plt.plot(final_records["Seed"],150*final_records["Score_diff_norm"])
#plt.xlabel("Team Playoff Seed")
#plt.ylabel("Team Season Score differential")
#plt.title("Playoff Seed vs Score differential")

In [58]:

#plt.plot(final_records["Seed"],final_records["Score_diff_norm"])
#plt.xlabel("Team Playoff Seed")
#plt.ylabel("Team Season Score differential")
#plt.title("Playoff Seed vs Score differential")

In [59]:
# Define the structure of the NumPy array
dtype = [('Team', 'U50'), ('Elo', 'f8')]

# Create an empty NumPy array with the defined structure
elo_table = np.empty(0, dtype=dtype)
print(elo_table)

[]


In [60]:
c = 400 # some parameter (cannonical value?)
K = 20
start_elo = 1500
for game in range (0,31): #should be 31 games but #19 wisconsin has incomplete data
    for team in range (1,67): #67
        # grab game row
        current_table = game_records[team]
        # debug print statements
        # print(team)
        # print(game)
        team_name = final_records['Team'][team]
        team_elo = start_elo
        opponent_name = current_table["Opponent"][game]
        opponent_elo = start_elo
        # read opp elo
        if (opponent_name in elo_table['Team']):
            opponent_elo = elo_table[elo_table['Team'] == opponent_name]['Elo'][0]
        else:
            elo_table = np.append(elo_table, np.array([(opponent_name,start_elo)], dtype=dtype))
        # read team elo
        if (team_name in elo_table['Team']):
            team_elo = elo_table[elo_table['Team'] == team_name]['Elo'][0]
        else:
            elo_table = np.append(elo_table, np.array([(team_name,start_elo)], dtype=dtype))
        
        # update team elo
        E = 1/(1+10**((opponent_elo - team_elo)/c)) # expected chance of win
        ratio = current_table['Score_diff_ratio'][game]    
        
        # handle edge case 
        if(current_table['W/L'][game] == 'T'):
            continue
        if(current_table['W/L'][game] == 'W'):
            O = 1
            team_elo = team_elo + K * (O - E) * ratio
        else:
            O = 0
            team_elo = team_elo + K * (O - E) * ratio
        
        # Find the index of the team in the Elo table
        index = np.where(elo_table['Team'] == team_name)[0][0]
        # Update the Elo rating
        elo_table[index] = (team_name, team_elo) 

        # update opponent elo
        E = 1/(1+10**((team_elo - opponent_elo)/c)) # expected chance of win
        ratio = current_table['Score_diff_ratio'][game]    
        
        # handle edge case 
        if(current_table['W/L'][game] == 'T'):
            continue
        if(current_table['W/L'][game] == 'L'):
            O = 1
            opponent_elo = opponent_elo + K * (O - E) * ratio
        else:
            O = 0
            opponent_elo = opponent_elo + K * (O - E) * ratio
        
        # Find the index of the team in the Elo table
        index = np.where(elo_table['Team'] == opponent_name)[0][0]
        # Update the Elo rating
        elo_table[index] = (opponent_name, opponent_elo) 

print(elo_table)

[('Northern Arizona', 1461.2941088 ) ('Houston', 1761.59011632)
 ('Louisiana-Monroe', 1467.57595365) ('Purdue', 1784.43960061)
 ('Samford', 1611.29438754) ('North Carolina', 1740.35834291)
 ('Radford', 1495.99115394) ('Tennessee', 1693.0590379 )
 ('Tennessee Tech', 1465.72611501) ('Arizona', 1704.14416549)
 ('Morgan State', 1437.34145176) ('Marquette', 1629.50995276)
 ('Northern Illinois', 1458.65986681) ('Iowa St.', 1619.09182791)
 ('Green Bay', 1494.22915873) ('Baylor', 1655.12247101)
 ('Auburn', 1602.12122965) ('Creighton', 1636.17443061)
 ('Florida A&M', 1408.89297983) ('Kentucky', 1651.82493841)
 ('New Mexico State', 1457.6406763 ) ('Illinois', 1647.22974194)
 ('Eastern Illinois', 1450.16395399) ('Duke', 1655.43025712)
 ('Dartmouth', 1454.29748204) ('Kansas', 1652.15028939)
 ('North Carolina Central', 1476.84562266) ('Alabama', 1639.67941711)
 ('Morehead State', 1473.09859169) ('BYU', 1623.16999266)
 ('Houston Christian', 1435.04908354) ('San Diego St.', 1614.77355878)
 ('Cal Stat